In [1]:
import torch                                                                                                                                   
import torch.nn as nn                                                                                                                          
import torch.nn.functional as F                                                                                                                
import torchvision                                                                                                                             
from torch.utils.data import DataLoader                                                                                                        
                                                                                                                                                
# Add parent dir to path if running from cifar_exp/                                                                                            
import sys                                                                                                                                     
from pathlib import Path                                                                                                                       
sys.path.insert(0, str(Path.cwd().parent) if Path.cwd().name == 'cifar_exp' else str(Path.cwd()))                                              
                                                                                                                                                
from layers import lorentz_resnet18, Lorentz                                                                                                   

In [2]:
manifold = Lorentz(k=1.0)

In [3]:
model = lorentz_resnet18(                                                                                                                      
    num_classes=100,                                                                                                                           
    base_dim=64,                                                                                                                               
    manifold=manifold,                                                                                                                         
    activation=nn.ReLU,                                                                                                                        
    init_method='lorentz_kaiming',                                                                                                             
)

In [4]:
total_els = 0
for n, p in model.named_parameters():
    print(n, p.numel())
    total_els += p.numel()
print(total_els)

manifold.c_softplus_inv 1
input_proj.conv.weight 192
input_proj.conv.bias 64
stage1.0.alpha_logit 1
stage1.0.layer1.fc.U 36864
stage1.0.layer1.fc.a 64
stage1.0.layer1.fc.V_auxiliary 36928
stage1.0.bn1.gamma 64
stage1.0.bn1.beta_space 64
stage1.0.layer2.fc.U 36864
stage1.0.layer2.fc.a 64
stage1.0.layer2.fc.V_auxiliary 36928
stage1.0.bn2.gamma 64
stage1.0.bn2.beta_space 64
stage1.1.alpha_logit 1
stage1.1.layer1.fc.U 36864
stage1.1.layer1.fc.a 64
stage1.1.layer1.fc.V_auxiliary 36928
stage1.1.bn1.gamma 64
stage1.1.bn1.beta_space 64
stage1.1.layer2.fc.U 36864
stage1.1.layer2.fc.a 64
stage1.1.layer2.fc.V_auxiliary 36928
stage1.1.bn2.gamma 64
stage1.1.bn2.beta_space 64
stage2.0.alpha_logit 1
stage2.0.layer1.fc.U 73728
stage2.0.layer1.fc.a 128
stage2.0.layer1.fc.V_auxiliary 73856
stage2.0.bn1.gamma 128
stage2.0.bn1.beta_space 128
stage2.0.layer2.fc.U 147456
stage2.0.layer2.fc.a 128
stage2.0.layer2.fc.V_auxiliary 147584
stage2.0.bn2.gamma 128
stage2.0.bn2.beta_space 128
stage2.0.proj.fc.U 8192


In [ ]:
import torch                                                                                                                                   
import torch.nn as nn                                                                                                                          
import torch.nn.functional as F                                                                                                                
import torchvision                                                                                                                             
from torch.utils.data import DataLoader                                                                                                        
                                                                                                                                                
# Add parent dir to path if running from cifar_exp/                                                                                            
import sys                                                                                                                                     
from pathlib import Path                                                                                                                       
sys.path.insert(0, str(Path.cwd().parent) if Path.cwd().name == 'cifar_exp' else str(Path.cwd()))                                              
                                                                                                                                                
from layers import lorentz_resnet18, Lorentz                                                                                                   
                                                                                                                                                
# Config                                                                                                                                       
device = 'cuda'                                                                                                                                
batch_size = 128                                                                                                                               
lr = 1e-1                                                                                                                                      
num_epochs = 50                                                                                                                                
                                                                                                                                                
# Data                                                                                                                                         
mean, std = (0.5074, 0.4867, 0.4411), (0.267, 0.256, 0.276)                                                                                    
train_transform = torchvision.transforms.Compose([                                                                                             
    torchvision.transforms.RandomCrop(32, padding=4),                                                                                          
    torchvision.transforms.RandomHorizontalFlip(),                                                                                             
    torchvision.transforms.ToTensor(),                                                                                                         
    torchvision.transforms.Normalize(mean, std)                                                                                                
])                                                                                                                                             
test_transform = torchvision.transforms.Compose([                                                                                              
    torchvision.transforms.ToTensor(),                                                                                                         
    torchvision.transforms.Normalize(mean, std)                                                                                                
])                                                                                                                                             
                                                                                                                                                
trainset = torchvision.datasets.CIFAR100('./data', train=True, download=True, transform=train_transform)                                       
testset = torchvision.datasets.CIFAR100('./data', train=False, download=True, transform=test_transform)                                        
train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)                                                        
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)                                                         
                                                                                                                                                
# Model                                                                                                                                        
manifold = Lorentz(k=1.0)                                                                                                                      
model = lorentz_resnet18(                                                                                                                      
    num_classes=100,                                                                                                                           
    base_dim=64,                                                                                                                               
    manifold=manifold,                                                                                                                         
    activation=nn.ReLU,                                                                                                                        
    init_method='lorentz_kaiming',                                                                                                             
).to(device)
# model.compile()                                                                                                                                   
                                                                                                                                                
# Optimizer                                                                                                                                    
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-5, nesterov=True)                                                      
                                                                                                                                                
# Training loop                                                                                                                                
def train_epoch(model, loader, optimizer):                                                                                                     
    model.train()                                                                                                                              
    total_loss, correct, total = 0, 0, 0                                                                                                       
    for x, y in loader:                                                                                                              
        x, y = x.to(device), y.to(device)     
        print(x)                                                                                                 
        optimizer.zero_grad()                                                                                                                  
        logits = model(x).squeeze()                                                                                                            
        loss = F.cross_entropy(logits, y)                                                                                                      
        loss.backward()                                                                                                                        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)                                                                                
        optimizer.step()                                                                                                                       
        total_loss += loss.item() * x.size(0)                                                                                                  
        correct += (logits.argmax(1) == y).sum().item()                                                                                        
        total += x.size(0)                                                                                                                     
    return total_loss / total, correct / total                                                                                                 
                                                                                                                                                
@torch.no_grad()                                                                                                                               
def evaluate(model, loader):                                                                                                                   
    model.eval()                                                                                                                               
    correct, total = 0, 0                                                                                                                      
    for x, y in loader:                                                                                                                        
        x, y = x.to(device), y.to(device)                                                                                                      
        logits = model(x).squeeze()                                                                                                            
        correct += (logits.argmax(1) == y).sum().item()                                                                                        
        total += x.size(0)                                                                                                                     
    return correct / total                                                                                                                     
                                                                                                                                                
# Train                                                                                                                                        
for epoch in range(num_epochs):                                                                                                                
    train_loss, train_acc = train_epoch(model, train_loader, optimizer)                                                                        
    test_acc = evaluate(model, test_loader)                                                                                                    
    print(f"Epoch {epoch+1}: train_loss={train_loss:.4f}, train_acc={train_acc:.4f}, test_acc={test_acc:.4f}")      

tensor([[[[ 0.1853,  0.1412,  0.1999,  ..., -1.9004, -1.9004, -1.9004],
          [ 0.1999,  0.2587,  0.2587,  ..., -1.9004, -1.9004, -1.9004],
          [-0.0938,  0.2293,  0.0531,  ..., -1.9004, -1.9004, -1.9004],
          ...,
          [-1.9004, -1.9004, -1.9004,  ..., -1.9004, -1.9004, -1.9004],
          [-1.9004, -1.9004, -1.9004,  ..., -1.9004, -1.9004, -1.9004],
          [-1.9004, -1.9004, -1.9004,  ..., -1.9004, -1.9004, -1.9004]],

         [[ 0.3047,  0.3047,  0.2894,  ..., -1.9012, -1.9012, -1.9012],
          [ 0.2588,  0.4273,  0.3813,  ..., -1.9012, -1.9012, -1.9012],
          [-0.1395,  0.3200,  0.2434,  ..., -1.9012, -1.9012, -1.9012],
          ...,
          [-1.9012, -1.9012, -1.9012,  ..., -1.9012, -1.9012, -1.9012],
          [-1.9012, -1.9012, -1.9012,  ..., -1.9012, -1.9012, -1.9012],
          [-1.9012, -1.9012, -1.9012,  ..., -1.9012, -1.9012, -1.9012]],

         [[ 0.2205,  0.2063,  0.2916,  ..., -1.5982, -1.5982, -1.5982],
          [ 0.1779,  0.3058,  

KeyboardInterrupt: 